In [1]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from skimage import feature


In [2]:
def getBorderColor(img):
    
    border = np.asarray(img[0,:])
    border = np.concatenate((border, np.asarray(img[-1, :])))
    border = np.concatenate((border, np.asarray(img[:, 0])))
    border = np.concatenate((border, np.asarray(img[:, -1])))
    #print(border)
    #print(border)
    return np.bincount(border).argmax()

def pre_processing(img):
    #img_blurred = cv2.medianBlur(img,5)
    #img_binarized = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    _, img_binarized = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)    
    if(getBorderColor(img_binarized) != 0):
        img_binarized = cv2.bitwise_not(img_binarized) 
    #img_binarized = adaptiveThresh(img, t=15, div=8)
    return img_binarized

In [3]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(pre_processing(img))
    return images

In [4]:
#follow the ration 60% 20% 20%
TRAINING_SIZE = 114
VALIDATION_SIZE = 38
TEST_SIZE = 38

TRAINING_OFFSET = TRAINING_SIZE
VALIDATION_OFFSET = TRAINING_OFFSET + VALIDATION_SIZE

training_set = []
validation_set = []
test_set = []

for i in range (1, 10):
    class_i_images = load_images_from_folder("ACdata_base/" + str(i))
    training_set.append(class_i_images[0:TRAINING_OFFSET])
    validation_set.append(class_i_images[TRAINING_OFFSET: VALIDATION_OFFSET])
    test_set.append(class_i_images[VALIDATION_OFFSET:])

In [5]:

def showImage(img):
    plt.imshow(img, cmap='gray')
    plt.show()
    #print(getBorderColor(img))
    #w, h = img.shape[:2]
    #print(cv2.countNonZero(img)/(w*h) * 100)


#arr = np.array([[1,2,3], [4,5,6], [7,8,9]])
#getBorderColor(arr)

In [6]:

# for i in range(len(test_set)):
#     for j in range(len(test_set[i])):
#         showImage(test_set[i][j])

In [8]:
def getFeaturesHOG(img):
   img = cv2.resize(img, (256, 128))
   cell_size = (32, 32)  # h x w in pixels
   block_size = (2, 2)  # h x w in cells
   nbins = 9  # number of orientation bins
   # winSize is the size of the image cropped to an multiple of the cell size
   hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1],
                                    img.shape[0] // cell_size[0] * cell_size[0]),
                            _blockSize=(block_size[1] * cell_size[1],
                                        block_size[0] * cell_size[0]),
                            _blockStride=(cell_size[1], cell_size[0]),
                            _cellSize=(cell_size[1], cell_size[0]),
                            _nbins=nbins)

   hog_feats = hog.compute(img)
   return hog_feats.flatten() 

(760,)


In [9]:
def get_X_Y_Train():
   X_Train = []
   Y_Train = []
   for i in range(len(training_set)):
      for j in range(len(training_set[i])):
        X_Train.append(getFeaturesHOG(training_set[i][j]))
        Y_Train.append(i+1)
   return (X_Train, Y_Train)

In [30]:
img = training_set[0][0]
print(len(getFeaturesHOG(img)))

[0.0770224  0.07650681 0.13061818 0.01054455 0.03472795 0.00516645
 0.01958773 0.01503969 0.05778803 0.27643016 0.17643142 0.23880607
 0.0433565  0.12003662 0.04115885 0.09628548 0.06845025 0.27643016
 0.27643016 0.2148967  0.23600875 0.03468615 0.09649448 0.03088595
 0.06002588 0.05562139 0.2234093  0.27643016 0.27643016 0.27643016
 0.07495617 0.2306774  0.08186118 0.23113815 0.1385377  0.27643016
 0.27691466 0.11829367 0.08608106 0.00999536 0.06678373 0.01760674
 0.05702645 0.04779318 0.2355729  0.28247795 0.28247795 0.19149747
 0.01178621 0.08186948 0.01028627 0.06968152 0.07163569 0.28247795
 0.28247795 0.1446061  0.15137187 0.01380011 0.08622541 0.03493083
 0.11615429 0.07851248 0.28247795 0.28247795 0.2503648  0.18183537
 0.01053722 0.08462914 0.03179661 0.15222023 0.14663392 0.28247795
 0.29705027 0.29705027 0.29705027 0.04205282 0.13263135 0.0115477
 0.08351371 0.087493   0.29705027 0.13224617 0.12009529 0.18092231
 0.03990197 0.13049915 0.00893097 0.03417914 0.02615156 0.10485